In [0]:
--CREATE CATALOG SPARK_SQL

select * from spark_sql.ecommerce.ecomm_orders
WHERE product_category='Fashion';


select 
  product_category,
  month(order_date) as Order_Month,
  count(order_id) as orders 
from 
  spark_sql.ecommerce.ecomm_orders
group by 
  order_month, product_category
having
  product_category = 'Fashion'
order by 
  order_month, orders desc;


--- Sub Queries
select * from(
  select * from (
    select 
      product_category,
      month(order_date) as Order_Month,
      count(order_id) as orders 
    from 
      spark_sql.ecommerce.ecomm_orders
    group by 
      order_month, product_category
    order by 
      order_month, orders desc
     ) tbl1
      where product_category = 'Fashion'  
  ) tbl2
where order_month = 5



--- CASE Statements
SELECT * FROM (
SELECT
  *,
  CASE  
    WHEN (Payment_method LIKE '%Card%') AND (Order_status in ('Cancelled', 'Returned')) THEN 'CARD'
    WHEN (Payment_method IN ('UPI','PayPal')) AND (Order_status in ('Cancelled', 'Returned')) THEN 'CASH'
    ELSE 'NO VALUE'
  END AS Payment_Flag
  FROM
    spark_sql.ecommerce.ecomm_orders ) flagged
  where 
  Payment_Flag = 'CASH';
  


  --- COMMON TABLE EXPRESSIONS
WITH CTE_TABLE_MAIN AS
(
  WITH CTE_TABLE AS (
    SELECT
      *,
      CASE  
        WHEN (Payment_method LIKE '%Card%') AND (Order_status in ('Cancelled', 'Returned')) THEN 'CARD'
        WHEN (Payment_method IN ('UPI','PayPal')) AND (Order_status in ('Cancelled', 'Returned')) THEN 'CASH'
        ELSE 'NO VALUE'
      END AS Payment_Flag
    FROM
      spark_sql.ecommerce.ecomm_orders
    )
  SELECT * FROM CTE_TABLE 
    where Payment_Flag = 'CASH'
)
SELECT 
  * 
  FROM 
    CTE_TABLE_MAIN
  WHERE
    product_category = 'Electronics'


--- WINDOW Functions 
SELECT
  price_per_unit,
  RANK() OVER (ORDER BY price_per_unit DESC) AS price_rank,
  DENSE_RANK() OVER (ORDER BY price_per_unit DESC) AS dense_rank,
  ROW_NUMBER() OVER (ORDER BY price_per_unit DESC) AS row_number
FROM
  spark_sql.ecommerce.ecomm_orders;

select sum(price_per_unit) as total_price from spark_sql.ecommerce.ecomm_orders


----FRAME CLAUSES
---sum applied on all rows
select 
  *,
  sum(price_per_unit) over (order by price_per_unit rows between unbounded preceding and unbounded following) as total_price 
from 
  spark_sql.ecommerce.ecomm_orders


---Running Total
select 
  *,
  sum(price_per_unit) over (order by price_per_unit rows between unbounded preceding and current row) as total_price 
from 
  spark_sql.ecommerce.ecomm_orders